In [ ]:
!pip install rdflib

from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, XSD

import csv

In [ ]:
path_csv = '/content/300356-0-monumentos-ciudad-madrid.csv'
path_ttl = '/content/monuments.ttl'

g = Graph()

g.parse(path_ttl, format='turtle')

print(f"Tripletas cargadas: {len(g)}")

NS = Namespace("http://www.semanticweb.org/vic/ontologies/2024/10/monuments/")

subtype_mapping = {
    "Escultura conceptual o abstracta": NS.AbstractSculpture,
    "Grupo Escultórico": NS.SculpturalGroup,
    "Elemento conmemorativo, Lápida": NS.MemorialElement,
    "Estatua": NS.Statue,
    "Puente, construcción civil": NS.CivilConstruction,
    "Fuente, Estanque, Lámina de agua": NS.Fountain,
    "Elemento de ornamentación": NS.Ornamentation,
    "Edificación singular": NS.UniqueBuilding,
    "Puerta, Arco triunfal": NS.Gate
}

with open(path_csv, 'r', encoding='latin-1') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=';')
    for row in reader:
        # Determinar la clase del monumento
        monument_class = subtype_mapping.get(row['SUBTIPO'], NS.Monument)  # Por defecto, Monument
        # Crear un URI único para el monumento
        monument_uri = URIRef(NS[f"{monument_class}_{row['PK']}"])

        g.add((monument_uri, RDF.type, monument_class))

        # Añadir propiedades
        g.add((monument_uri, NS.PK, Literal(row['PK'], datatype=XSD.string)))
        g.add((monument_uri, NS.name, Literal(row['NOMBRE'], datatype=XSD.string)))
        g.add((monument_uri, NS.description, Literal(row['DESCRIPCION'], datatype=XSD.string)))
        g.add((monument_uri, NS.date, Literal(row['FECHA'], datatype=XSD.string)))

        # Añadir geometría
        geometry_uri = URIRef(NS[f"Geometry_{row['PK']}"])
        g.add((geometry_uri, RDF.type, NS.Geometry))
        g.add((geometry_uri, NS.geo_asWKT, Literal(f"POINT({row['LONGITUD']} {row['LATITUD']})", datatype=XSD.string)))
        g.add((monument_uri, NS.hasGeometry, geometry_uri))

         # Añadir información de autores (si hay múltiples, separarlos)
        if row['AUTORES']:
            authors = row['AUTORES'].split(" - ")
            for author in authors:
                author_uri = URIRef(NS[f"Author_{author.replace(' ', '_')}"])
                g.add((author_uri, RDF.type, NS.Author))
                g.add((author_uri, NS.name, Literal(author.strip(), datatype=XSD.string)))
                g.add((monument_uri, NS.author, author_uri))

        # Añadir URLs (PDF y contenido web)
        if row['PDF']:
            g.add((monument_uri, NS.infoURL, Literal(row['PDF'], datatype=XSD.anyURI)))
        if row['CONTENT-URL']:
            g.add((monument_uri, NS.infoURL, Literal(row['CONTENT-URL'], datatype=XSD.anyURI)))


print(f"Tripletas después de la importación: {len(g)}")

g.serialize(destination="/content/populated_monuments.ttl", format="turtle")


Tripletas cargadas: 161
Tripletas después de la importación: 21032


<Graph identifier=Nc5aecb8d3bb14f0aa466504fee3f4997 (<class 'rdflib.graph.Graph'>)>